In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import os
import sys
import raillabel

sys.path.insert(1, '/workspaces/baseline')
from pointcept.datasets import transform


In [2]:
OSDaR_path = "/workspaces/baseline/data/OSDaR_dataset/v_2"

In [3]:
train_set=["1_calibration_1.2","3_fire_site_3.1","3_fire_site_3.3","4_station_pedestrian_bridge_4.3","5_station_bergedorf_5.1","6_station_klein_flottbek_6.2","8_station_altona_8.1","8_station_altona_8.2","9_station_ruebenkamp_9.1","12_vegetation_steady_12.1","14_signals_station_14.1","15_construction_vehicle_15.1","20_vegetation_squirrel_20.1","21_station_wedel_21.1","21_station_wedel_21.2"]

In [4]:
[os.path.join(OSDaR_path, training_scene) for training_scene in train_set]

['/workspaces/baseline/data/OSDaR_dataset/v_2/1_calibration_1.2',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/3_fire_site_3.1',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/3_fire_site_3.3',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/4_station_pedestrian_bridge_4.3',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/5_station_bergedorf_5.1',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/6_station_klein_flottbek_6.2',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/8_station_altona_8.1',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/8_station_altona_8.2',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/9_station_ruebenkamp_9.1',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/12_vegetation_steady_12.1',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/14_signals_station_14.1',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/15_construction_vehicle_15.1',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/20_vegetation_squirrel_20.1',
 '/workspaces/baseline/data/OSDaR_dataset/v_2/21_station_wedel_21.1

In [5]:


training_dirs = [os.path.join(OSDaR_path, training_scene) for training_scene in train_set]

result_dict = {"nb_pts_tot":[], "nb_pts_sampled":[], "scene":[], "frame":[]}

i = 1
for folder_dir in training_dirs: # Iterate through each training scene
    lidar_dir = os.path.join(folder_dir, "lidar")

    for file in os.listdir(lidar_dir):
        lidar_frame_path = os.path.join(lidar_dir, file)

        with open(lidar_frame_path, "r") as pcd:
            scan = np.loadtxt(pcd, skiprows=11, usecols=(0,1,2,3))
        coord = scan[:, :3]     # The x,y,z coordinates of the points 
        
        input_dict = {"coord": coord} 

        grid_sampling = transform.GridSample(keys=["coord"]) 

        grid_sampled_dict = grid_sampling(input_dict)

        result_dict["nb_pts_tot"].append(len(coord))
        result_dict["nb_pts_sampled"].append(len(grid_sampled_dict["coord"]))
        result_dict["scene"].append(folder_dir.rsplit("/",1)[1])
        result_dict["frame"].append(file.split("_")[0])
        
    print(f"{i}/{len(training_dirs)} done.")
    i+=1


1/15 done.
2/15 done.
3/15 done.
4/15 done.
5/15 done.
6/15 done.
7/15 done.
8/15 done.
9/15 done.
10/15 done.
11/15 done.
12/15 done.
13/15 done.
14/15 done.
15/15 done.


In [6]:
df_pts = pd.DataFrame.from_dict(result_dict)

In [7]:
df_pts_per_scene=df_pts.groupby("scene")[["nb_pts_tot", "nb_pts_sampled"]].mean().reset_index()


In [20]:
fig = px.bar(df_pts_per_scene.sort_values(by=["nb_pts_sampled"],ascending=False), x="scene", y=["nb_pts_tot", "nb_pts_sampled"],barmode = 'group',title="Average number of points per scene in training set")
fig.update_layout(xaxis_title="Scene", yaxis_title="Average number of points",autosize=False,
    width=1000,
    height=600,
    margin=dict(l=20, r=20, t=30, b=20),)


In [21]:
fig.write_image("/workspaces/baseline/exp/figures/nb_pts_per_scene.png")